In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from RDMFS import HF_U2RDM, MU_U2RDM, GU_U2RDM, BBC1_U2RDM
from utils import R_spectral_clean, DM2_DiagSum, twoDM_Eigvals, BST_FIDX, spectral_clean
import scipy 
import math

In [2]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 1.5
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
H    {L}    {L}   0.0
H   {-L}    {L}   0.0
"""
# this basis has 2 functions for Helium
# mol.basis = "ccpvdz"
mol.basis = "sto-3g"
# mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[1,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b
h1_mo = (h1_a, h1_b)


h2_mo = BST_FIDX((eri_ao, eri_ao, eri_ao),S@C_a,S@C_b)




In [3]:
dm1_ao = mf.make_rdm1()
dm2_ao = mf.make_rdm2()

dm1_mo = dm1_ao.copy()
# # dm1_mo=C[:,0:N]@dm1_ao@C[:,0:N].T
dm1_mo[0]=(C_a.T@(S.T@dm1_mo[0]@S))@C_a
dm1_mo[1]=(C_a.T@(S.T@dm1_mo[1]@S))@C_a

dm2_mo = BST_FIDX(dm2_ao,S@C_a,S@C_b)

# P = C@C.T
# print(P)

occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_mo, h1.shape[0])
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))


[1.0000000e+00 1.0000000e+00 1.1596858e-16 0.0000000e+00] 1.9999999999999993
[1.00000000e+00 1.00000000e+00 8.80978257e-17 0.00000000e+00] 1.9999999999999993


In [4]:
dm2_HF_mo = HF_U2RDM(occ_mo_a, occ_mo_b, mol.nao)

In [5]:
L_CONCERN = False
if L_CONCERN:
    for i in range(0,mol.nao):
        for j in range(0,mol.nao):
            for k in range(0,mol.nao):
                for l in range(0,mol.nao):
                    if not math.isclose(dm2_HF_mo[0][i,j,k,l], dm2_mo[0][i,j,k,l], abs_tol = 1e-10):
                        print(i,j,k,l, dm2_HF_mo[0][i,j,k,l], dm2_mo[0][i,j,k,l] )
    for i in range(0,mol.nao):
        for j in range(0,mol.nao):
            for k in range(0,mol.nao):
                for l in range(0,mol.nao):
                    if not math.isclose(dm2_HF_mo[1][i,j,k,l], dm2_mo[1][i,j,k,l], abs_tol = 1e-10):
                        print(i,j,k,l, dm2_HF_mo[1][i,j,k,l], dm2_mo[1][i,j,k,l])
    for i in range(0,mol.nao):
        for j in range(0,mol.nao):
            for k in range(0,mol.nao):
                for l in range(0,mol.nao):
                    if not math.isclose(dm2_HF_mo[2][i,j,k,l], dm2_mo[2][i,j,k,l], abs_tol = 1e-10):
                        print(i,j,k,l, dm2_HF_mo[2][i,j,k,l], dm2_mo[2][i,j,k,l] )

0 0 0 3 0.0 3.736584101047309e-06
0 0 1 2 0.0 1.171706257177095e-05
0 0 2 1 0.0 1.1717062571755257e-05
0 0 2 2 8.80978256731728e-17 1.372895370901281e-10
0 0 3 0 0.0 3.7365841010547473e-06
0 3 0 0 0.0 3.7365841007629114e-06
0 3 1 1 0.0 3.7365841002841527e-06
1 1 0 3 0.0 3.736584100588321e-06
1 1 1 2 0.0 1.17170625703266e-05
1 1 2 1 0.0 1.1717062570322502e-05
1 1 2 2 8.809782567317272e-17 1.3728953509797533e-10
1 1 3 0 0.0 3.7365841006006474e-06
1 2 0 0 0.0 1.1717062571325755e-05
1 2 1 1 0.0 1.1717062569879775e-05
1 2 1 2 0.0 1.3728955211978974e-10
1 2 2 1 0.0 1.37289551689799e-10
2 1 0 0 0.0 1.171706257129136e-05
2 1 1 1 0.0 1.1717062569897553e-05
2 1 1 2 0.0 1.3728955212007912e-10
2 1 2 1 0.0 1.3728955168944336e-10
2 2 0 0 1.159685796530515e-16 1.3728952703485306e-10
2 2 1 1 1.1596857965305133e-16 1.3728954170395028e-10
3 0 0 0 0.0 3.7365841007853205e-06
3 0 1 1 0.0 3.7365841003118643e-06


In [10]:
print(np.sum(twoDM_Eigvals(dm2_HF_mo)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_mo)), scipy.special.binom(4,2))

# print(np.sum(twoDM_Eigvals(dm2_HF_mo)), scipy.special.binom(4,2))
# print(np.sum(twoDM_Eigvals(dm2_mo)), scipy.special.binom(4,2))

11.999999999999993 6.0
11.999999999999984 6.0


In [11]:
# Run FCI  
cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e_mo, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
dm1_fci, dm2_fci  = cisolver.make_rdm12s(ci_mo, h1.shape[0], mol.nelec)
occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_fci, h1.shape[0])
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))


[0.98295595 0.5        0.5        0.01704405] 2.000000000000001
[0.98295595 0.5        0.5        0.01704405] 2.0000000000000013


In [12]:
dm2_HF_fci = HF_U2RDM(occ_mo_a, occ_mo_b, mol.nao)

In [13]:
print(np.sum(twoDM_Eigvals(dm2_HF_fci)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_fci)), scipy.special.binom(4,2))

13.067014208521433 6.0
12.000000000000009 6.0
